In [18]:
#Q1

from os import system
from IPython.display import clear_output
import copy
import queue as PQ


class state:
    #state class for maintaining states

    def __init__(self,position,previous_position,move,dep,size,cost):
        self.position=position                  #array of position
        self.parent=previous_position           #store the parent node
        self.depth=dep                          #store the depth
        self.dimension=size                     #store the dimension of array
        self.last_move=move                     #store the last move made
        self.cost=cost                          #store the node cost


class comparator:
    #custom comparator for priority queue
    def __init__(self,cost,state):
        self.cost=cost
        self.state=state

    #condtition used to assign priorties in pq
    def __lt__(self, other):
        return self.cost < other.cost

    
    def __eq__(self, other):
        return self.state.position==other.state.position

def print_path(node):
    while(node!=None):
        print("Cost: ",node.cost)
        print_puzzle(node)
        node=node.parent

def print_puzzle(node):

    #print the state position
    print('-----------------------------------')
    for i in range(0,len(node.position)):
        if(i%node.dimension==0 and i!=0):
            print('\n')
        print(node.position[i],end=" ")
    print('\n-----------------------------------')

def menu():
    print("1.UCS\n2.Greedy(Manhattan)\n3.Greedy(Misplaced Tiles)\n4.Astar(Manhattan)\n5.Astar(Misplaced Tiles)")


def getInvCount(arr,n):
    inv_count = 0
    for i in range(0, n**2):
        if(arr[i])==-1:
            continue
        for j in range(i+1,n**2):
            if(arr[i]>arr[j] and arr[i]!=-1):
                inv_count+=1
        
    return inv_count

def isSolvable(puzzle,n) :
 
    # Count inversions in given 8 puzzle
    inv_count = getInvCount(puzzle,n)
 
    # return true if inversion count is even.
    return (inv_count % 2 == 0)


def moveup(node):
    #return the state after moving space if possible

    new=copy.deepcopy(node.position)        #deepcopy of array
    space_index=new.index(-1)               #find the space tile
    for i in range(0,node.dimension):       #check the boundry conditions
        if(space_index==i):
            return None                     #return None if state not possible

    new[space_index],new[space_index-node.dimension]=new[space_index-node.dimension],new[space_index]       #swap the space with the up value
    return new

def movedown(node):
    new=copy.deepcopy(node.position)
    space_index=new.index(-1)               #find space tile
    length=len(new)
    for i in range(0,node.dimension):
        if(space_index==length-1-i):        #check the boundry conditions
            return None

    new[space_index],new[space_index+node.dimension]=new[space_index+node.dimension],new[space_index]       #swap space tile with down value
    return new


def moveright(node):
    new=copy.deepcopy(node.position)
    space_index=new.index(-1)               #find space tile
    interval=node.dimension-1
    for i in range(0,node.dimension):       #boundry conditions
        if(space_index==interval):
            return None
        interval+=node.dimension

    new[space_index],new[space_index+1]=new[space_index+1],new[space_index]     #swap space tile with right value
    return new

def moveleft(node):
    new=copy.deepcopy(node.position)
    space_index=new.index(-1)               #find space tile
    interval=0
    for i in range(0,node.dimension):       #boundry conditions
        if(space_index==interval):
            return None
        interval+=node.dimension

    new[space_index],new[space_index-1]=new[space_index-1],new[space_index]     #swap space tile with left value
    return new



def CreateAposition(node):
    #returns the list of new states

    new=node
    new_states=[]
    up_position=None                           #initilialize the positions
    down_position=None
    left_position=None
    right_position=None

    if(node.last_move!="down"):                #checking if not moving back to the state it came from else assign the new states
        up_position=moveup(new)                 
    if(node.last_move!="up"):
        down_position=movedown(new)
    if(node.last_move!="right"):
        left_position=moveleft(new)
    if(node.last_move!="left"):
        right_position=moveright(new)


    if(up_position!=None):                      #make object of the new position arrays
        new_states.append(state(up_position,node,"up",node.depth+1,node.dimension,node.cost+1))
    if(down_position!=None):
        new_states.append(state(down_position,node,"down",node.depth+1,node.dimension,node.cost+1))
    if(left_position!=None):
        new_states.append(state(left_position,node,"left",node.depth+1,node.dimension,node.cost+1))
    if(right_position!=None):
        new_states.append(state(right_position,node,"right",node.depth+1,node.dimension,node.cost+1))

    return new_states                           #return the list of new states


def UCS(i_node,g_node,size):
    initial_state=state(i_node,None,None,0,size,0)
    goal_state=state(g_node,None,None,0,size,0)

    queue=PQ.PriorityQueue()                                        #make the priority queue
    closed=[]                                                       #array to remove duplicated states

    queue.put(comparator(initial_state.cost,initial_state))         #store the states according to their cost as priority
    closed.append(initial_state.position)

    for i in range(100000):                                           #terminating condition to prevent generating infinite states and fill up the memory
        obj=queue.get()                                             #pop the top state

        # print(obj.state.cost)                                     uncomment to see position and cost at each postition but output is very large on console
        # print_puzzle(obj.state)
        if(obj.state.position==goal_state.position):                #goal condition
            print_path(obj.state)
            return True


        new_states=CreateAposition(obj.state)                       #return list of states
        for states in new_states:
            if  states not in closed:                               #if state already visited,do not append
                closed.append(states)
                queue.put(comparator(states.cost,states))

    return False                                                    #return false if goal not found

def Greedy_A(i_node,g_node,size):
    initial_state=state(i_node,None,None,0,size,0)
    goal_state=state(g_node,None,None,0,size,0)

    queue=PQ.PriorityQueue()
    closed=[]
    heuristic=Manhattan_distance(initial_state,goal_state)          #h(n)=manhattan distance
    initial_state.cost=heuristic
    queue.put(comparator(initial_state.cost,initial_state))         #store the states according to their h(n) values
    closed.append(initial_state.position)

    while(True):                                  
        if(queue.empty()):                                          #if queue becomes empty return false
            return False
        obj=queue.get()
        
        # print("Manhattan distance: ",obj.cost)                    uncomment to see position and cost at each postition but output is very large on console
        # print_puzzle(obj.state)
        if(obj.state.position==goal_state.position):                #goal condition
            print_path(obj.state)
            return True

        
        new_states=CreateAposition(obj.state)                       #return list of states
        for states in new_states:
            if  states.position not in closed:                      #check for duplicate states
                closed.append(states.position)
                heuristic=Manhattan_distance(states,goal_state)     #find the h(n) of new states and store them in pq
                states.cost=heuristic
                queue.put(comparator(states.cost,states))

    return False


def Greedy_B(i_node,g_node,size):
    initial_state=state(i_node,None,None,0,size,0)
    goal_state=state(g_node,None,None,0,size,0)

    queue=PQ.PriorityQueue()                                        #same as Greedy a but h(n)=misplaced tiles
    closed=[]
    heuristic=N_M_T(initial_state,goal_state)
    initial_state.cost=heuristic
    queue.put(comparator(initial_state.cost,initial_state))
    closed.append(initial_state.position)

    while(True):

        if(queue.empty()):
            return False
        obj=queue.get()

        # print("Misplaced tiles: ",obj.cost)                      uncomment to see position and cost at each postition but output is very large on console
        # print_puzzle(obj.state)
        if(obj.state.position==goal_state.position):
            print_path(obj.state)
            return True

      
        new_states=CreateAposition(obj.state)
        for states in new_states:
            if states.position not in closed:
                closed.append(states.position)
                heuristic=N_M_T(states,goal_state)
                states.cost=heuristic
                queue.put(comparator(states.cost,states))

    return False


def Astar_A(i_node,g_node,size):
    initial_state=state(i_node,None,None,0,size,0)                  #convert the initial and goal arrays into state objects
    goal_state=state(g_node,None,None,0,size,0)

    queue=PQ.PriorityQueue()                                        
    closed={}                                                       #check duplicate states iin dictionary
    heuristic=Manhattan_distance(initial_state,goal_state)          
    initial_state.cost=heuristic+initial_state.cost                 #f(n)=h(n)+g(n) where h(n)=Manhattan distance of a state

    queue.put(comparator(initial_state.cost,initial_state))         #store the states according to their f(n) cost as priority
    closed[str(initial_state.position)]=initial_state.cost

    while(True):                                       

        if(queue.empty()):
            return False
        obj=queue.get()                                             #pop the state with least cost

        # print("F(n): ",obj.state.cost)                            uncomment to see position and cost at each postition but output is very large on console
        # print_puzzle(obj.state)
        if(obj.state.position==goal_state.position):                #goal condition
            print_path(obj.state)
            return True


        new_states=CreateAposition(obj.state)                       #return list of states
        for states in new_states:
            heuristic=Manhattan_distance(states,goal_state)
            states.cost=heuristic+states.cost                       #calulate cost of new states

            if str(states.position) not in closed:                  #if state does not exist in duplicate array,append it
                queue.put(comparator(states.cost,states))
                closed[str(states.position)]=states.cost
            else:

                if(closed[str(states.position)]>states.cost):       #if state exists in duplicate array,compare the cost ,if cost of new state is less,push it in priority queue
                    closed[str(states.position)]=state.cost
                    queue.put(comparator(states.cost,states))

    return False


def Astar_B(i_node,g_node,size):
    initial_state=state(i_node,None,None,0,size,0)                  #same as Astar A but f(n)=h(n)+g(n) where f(N)=misplaced tiles
    goal_state=state(g_node,None,None,0,size,0)

    queue=PQ.PriorityQueue()
    closed={}
    heuristic=N_M_T(initial_state,goal_state)
    initial_state.cost=heuristic+initial_state.cost

    queue.put(comparator(initial_state.cost,initial_state))
    closed[str(initial_state.position)]=initial_state.cost

    itr=0
    while(True):

        if(queue.empty()):
            return False
        obj=queue.get()
        # print("F(n): ",obj.state.cost)                            uncomment to see position and cost at each postition but output is very large on console
        # print_puzzle(obj.state)
        
        if(obj.state.position==goal_state.position):
            print_path(obj.state)
            return True

      
        new_states=CreateAposition(obj.state)
        for states in new_states:
            heuristic=N_M_T(states,goal_state)
            states.cost=heuristic+states.cost

            if str(states.position) not in closed:
                queue.put(comparator(states.cost,states))
                closed[str(states.position)]=states.cost
            else:

                if(closed[str(states.position)]>states.cost):
                    closed[str(states.position)]=state.cost
                    queue.put(comparator(states.cost,states))

    return False

def Manhattan_distance(node,g_node):
    #return the manhattan distance of a state

    sum=0
    n=node.dimension
    for i in node.position:
        if(i==-1):
            continue
        node_val=node.position.index(i)                                 #find the state in goal state and initial state 
        g_val=g_node.position.index(i)
        sum+=abs(node_val%n - g_val%n) + abs(node_val//n - g_val//n)    #formula to find the distance in a single dimension array

    return sum                                                          #return the sum


def N_M_T(node,g_node):
    #return the number of misplaced tiles count

    sum=0
    for i in range (0,node.dimension**2):
        node_val=node.position[i]
        goal_val=g_node.position[i]
        if(node_val!=goal_val):                                         #if indexes are not same in initial and goal state,tile is misplaced
            sum+=1
    return sum                                                          #return the sum



def task1(n,initial_node,goal_node):
    if(UCS(initial_node,goal_node,n)==True):
        print("yes")
    else:
        print("no")


def task2_A(n,initial_node,goal_node):
    if(Greedy_A(initial_node,goal_node,n)==True):
        print("yes")
    else:
        print("no")

def task2_B(n,initial_node,goal_node):
    if(Greedy_B(initial_node,goal_node,n)==True):
        print("yes")
    else:
        print("no")


def task3_A(n,initial_node,goal_node):
    if(Astar_A(initial_node,goal_node,n)==True):
        print("yes")
    else:
        print("no")

def task3_B(n,initial_node,goal_node):
    if(Astar_B(initial_node,goal_node,n)==True):
        print("yes")
    else:
        print("no")


       
if __name__ == "__main__":
    n=int(input("Enter dimensions: "))
    initial_node=[]
    print("Enter initial state:")
    for i in range(0,n*n):
        inp=int(input("Enter a number"))
        initial_node.append(inp)
    
    print(initial_node)

    goal_node=[]
    print("Enter goal state:")
    for i in range(0,n*n):
        inp=int(input("Enter a number"))
        goal_node.append(inp)
    # print(goal_node)

    # n=3
    # initial_node=[1,2,3,4,5,6,8,7,-1]
    # goal_node=[6,2,8,-1,5,1,7,3,4]                  # solvable check only works for this goal state

    if(isSolvable(initial_node,n)) :
        print("Solvable")
    else :
        print("Not Solvable")

    while(1):
        menu()
        inp=int(input("\nEnter algorithm number:"))

    
        if(inp==1):
            task1(n,initial_node,goal_node)
        elif(inp==2):
            task2_A(n,initial_node,goal_node)
        elif(inp==3):
            task2_B(n,initial_node,goal_node)
        elif(inp==4):
            task3_A(n,initial_node,goal_node)
        elif(inp==5):
            task3_B(n,initial_node,goal_node)
        else:
            break
 

    
    

Not Solvable
1.UCS
2.Greedy(Manhattan)
3.Greedy(Misplaced Tiles)
4.Astar(Manhattan)
5.Astar(Misplaced Tiles)


KeyboardInterrupt: 

In [ ]:
# Q2
# (A)
# When a tree has only a single path to its goal, iterative deepening will perform worse than DFS. It is because dfs will keep moving to the next node without backtraking without reaching its goal which will take O(N).
# On the other hand, iterative deepening search will put a limit, traverse the limit (1 iteration) and then repeat for limit+1 (2 iterations) from the start until it reaches its goal.The sequence will be 
# 1+2+3+4+5.....N = 0(N^2)
# Hence Iterative deepening search will take more time

# (B) 

# (a)
# States: The arrangement of the track pieces connected together.
# Initial State: Any of the single selected track piece.
# Actions: Connect a loose piece to another loose piece or Remove a connected track piece.
# Goal: Connect all the pieces together without any loose holes or overlapping tracks.
# Path Cost: The cost of connecting or removing a piece is one.

# (b)
# As all the pieces have to be used and connected together, hence the tree formed will not have any shallow solutions(the goal node is not close to the root) and the goal will lie deep in the tree.
# So DFS should be used as it is most suitable for finding deep solutions and it also has a better space complexity compared to BFS and IDS. Since there are a limited number of pieces,
# it will not be stuck in an infinite loop and will be complete. DLS does not need to be applied as this problem has a finite set of states. Since step cost is one,
# Uniform cost search will work the same as BFS and perform worse than DFS.

# (c)
# Since there are an equal number of forks for starting and ending ends, removing any one of them will lead to inequality and there would always be an unconnected starting or loose end which will
# make it impossible for completing the railway track.





